# Maximum likelihood estimation with Kalman filter and automatic differentiation

### This notebook shows how to use `Kalman.jl` together with `ForwardDiff` to estimate the filter coefficients.


In [16]:
!haskey(Pkg.installed(), "GaussianDistributions") && Pkg.clone("https://github.com/mschauer/GaussianDistributions.jl")
!haskey(Pkg.installed(), "Kalman") && Pkg.clone("https://github.com/mschauer/Kalman.jl");

In [3]:
using Kalman
using StaticArrays

In [4]:
using ForwardDiff
using ForwardDiff: derivative, Dual

## Define a parametrized linear statespace model with parameter `θ`

In [ ]:
M(θ) = let x0 = 1.0,
    P0 = 1.0,

    Phi = θ,
    b = 0.0,
    Q = 1.0,

    y = NaN,
    H = 1.0,
    R = 1.0
    LinearHomogSystem(x0, P0, Phi, b, Q, y, H, R)
end

## Generate test data with `θ0 = 1.0`

In [5]:
θ0 = 1.0
M0 = M(θ0)

Kalman.LinearHomogSystem{Float64,Float64,Float64,Float64,Float64,Float64}(1.0, 1.0, 1.0, 0.0, 1.0, NaN, 1.0, 1.0)

In [20]:
n = 100
k = 10
Y, X0 = sample(n, k, M0)

([-0.705703 2.23168 … 1.50842 -1.4713; 0.673827 1.59258 … -1.08131 0.373971; … ; -13.3379 1.92383 … -1.1129 7.97751; -11.3865 -0.263185 … -1.97939 8.67953], [0.307742 1.15844 … 1.10713 -0.595886; 1.35578 1.06846 … -1.13856 -0.310503; … ; -12.6372 0.957971 … -0.539456 7.54155; -12.2364 -0.104968 … -1.23302 8.36806])

## Define iterator that sums the marginal log likelihood of the data

In [7]:
f(t, x, P, Ppred, ll, K) = ll
import Kalman: mappedreturntype
Kalman.mappedreturntype(_, ::Type{typeof(f)}) = Float64

In [8]:
target(x) = sum(sum(MappedKalmanFilter(zip(Base.Iterators.countfrom(1),view(Y, :, j)), M(x), f)) for j in 1:k)
target(v::SVector{1}) = target(v[1])

target (generic function with 2 methods)

## Fit `θ` using Newton with automatic derivatives

In [14]:
θ = 0.7
println(θ)
target2(v) = target(v[1])
for it in 1:10
    θold = θ
    θ = θ - inv(ForwardDiff.hessian(target2, SVector{1}(θ))[1])*ForwardDiff.derivative(target, θ)
    θold == θ && break    
    println(θ)
end
println("Maximum llikelihood estimate: θ = $θ")

0.7
1.119214625197172
0.9794681320562197
0.9878044690148431
0.9877983862786246
0.9877983862765932
Maximum llikelihood estimate: θ = 0.9877983862765932
